score: 1.0000

In [1]:
import time
import datetime as dt

import numpy as np
import pandas as pd

from tqdm import tqdm

tqdm.pandas()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df_orders = pd.read_csv('delivery_orders_march.csv')

In [3]:
def epoch_gmt8(X):
    return (dt.datetime.utcfromtimestamp(X) + dt.timedelta(hours=8)).date()

df_orders['2nd_deliver_attempt'] = df_orders['2nd_deliver_attempt'].replace(np.nan,0)
df_orders['pick_date'] = df_orders['pick'].progress_apply(epoch_gmt8)
df_orders['1st_attempt_date'] = df_orders['1st_deliver_attempt'].progress_apply(epoch_gmt8)
df_orders['2nd_attempt_date'] = df_orders['2nd_deliver_attempt'].progress_apply(epoch_gmt8)

100%|██████████| 3176313/3176313 [00:07<00:00, 410647.38it/s]


In [4]:
holidays = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']
weekmask = 'Mon Tue Wed Thu Fri Sat'

calendar = np.busdaycalendar(weekmask=weekmask, holidays=holidays)

In [5]:
def checkPickDate(X):
    while True:
        if np.is_busday(X, busdaycal=calendar):
            return X
        else:
            X += dt.timedelta(days=1)
            
df_orders['pick_date'] = df_orders['pick_date'].progress_apply(checkPickDate)

100%|██████████| 3176313/3176313 [00:21<00:00, 150191.73it/s]


In [6]:
def getRegion(X):
    X = X.split(' ')[-1].lower()
    
    if X == 'manila':
        return 'metro manila'
    else:
        return X

df_orders['buyer_region'] = df_orders['buyeraddress'].apply(getRegion)
df_orders['seller_region'] = df_orders['selleraddress'].apply(getRegion)

In [7]:
df_orders['1st_attempt_leadtime'] = np.busday_count(df_orders['pick_date'],
                                                    df_orders['1st_attempt_date'],
                                                    busdaycal=calendar)

df_orders['2nd_attempt_leadtime'] = np.busday_count(df_orders['1st_attempt_date'],
                                                    df_orders['2nd_attempt_date'],
                                                    busdaycal=calendar)

In [8]:
sla = {
    'metro manila': {'metro manila':3, 'luzon':5, 'visayas':7, 'mindanao':7},
    'luzon': {'metro manila':5, 'luzon':5, 'visayas':7, 'mindanao':7},
    'visayas': {'metro manila':7, 'luzon':7, 'visayas':7, 'mindanao':7},
    'mindanao': {'metro manila':7, 'luzon':7, 'visayas':7, 'mindanao':7}
}

def getSLA(X):
    return sla[X.seller_region][X.buyer_region]

df_orders['sla'] = df_orders.progress_apply(getSLA, axis=1)

100%|██████████| 3176313/3176313 [01:43<00:00, 30763.58it/s]


In [9]:
df_orders['isFirstLate'] = (df_orders['1st_attempt_leadtime'] > df_orders['sla'])
df_orders['is2ndLate'] = (df_orders['2nd_attempt_leadtime'] > 3)

In [10]:
df_orders['is_late'] = (df_orders['1st_attempt_leadtime'] > df_orders['sla']) | (df_orders['2nd_attempt_leadtime'] > 3)

In [11]:
df_orders[['orderid', 'is_late']].to_csv('submission.csv', index=False)